# Evidently Test Presets

In [ ]:
try:
    import evidently
except:
    !npm install -g yarn
    !pip install git+https://github.com/evidentlyai/evidently.git

In [ ]:
import pandas as pd
import numpy as np

from sklearn import datasets, ensemble, model_selection

from evidently import ColumnMapping
from evidently.test_suite import TestSuite

from evidently.test_preset import (NoTargetPerformance, DataQuality, DataStability, DataDrift,
Regression, MulticlassClassification, BinaryClassificationTopK, BinaryClassification)

## Prepare Datasets

In [ ]:
#Dataset for Data Quality and Integrity
adult_data = datasets.fetch_openml(name='adult', version=2, as_frame='auto')
adult = adult_data.frame

adult_ref = adult[~adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]
adult_cur = adult[adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]

adult_cur.iloc[:2000, 3:5] = np.nan

In [ ]:
#Dataset for Regression
housing_data = datasets.fetch_california_housing(as_frame='auto')
housing = housing_data.frame

housing.rename(columns={'MedHouseVal': 'target'}, inplace=True)
housing['prediction'] = housing_data['target'].values + np.random.normal(0, 3, housing.shape[0])

housing_ref = housing.sample(n=5000, replace=False)
housing_cur = housing.sample(n=5000, replace=False)

In [ ]:
#Dataset for Binary Probabilistic Classifcation
bcancer_data = datasets.load_breast_cancer(as_frame='auto')
bcancer = bcancer_data.frame

bcancer_ref = bcancer.sample(n=300, replace=False)
bcancer_cur = bcancer.sample(n=200, replace=False)

model = ensemble.RandomForestClassifier(random_state=1, n_estimators=10)
model.fit(bcancer_ref[bcancer_data.feature_names.tolist()], bcancer_ref.target)

bcancer_ref['prediction'] = model.predict_proba(bcancer_ref[bcancer_data.feature_names.tolist()])[:, 1]
bcancer_cur['prediction'] = model.predict_proba(bcancer_cur[bcancer_data.feature_names.tolist()])[:, 1]

In [ ]:
#Dataset for Multiclass Classifcation
iris_data = datasets.load_iris(as_frame='auto')
iris = iris_data.frame

iris_ref = iris.sample(n=75, replace=False)
iris_cur = iris.sample(n=75, replace=False)

model = ensemble.RandomForestClassifier(random_state=1, n_estimators=3)
model.fit(iris_ref[iris_data.feature_names], iris_ref.target)

iris_ref['prediction'] = model.predict(iris_ref[iris_data.feature_names])
iris_cur['prediction'] = model.predict(iris_cur[iris_data.feature_names])

## How to use Test Presets?

In [ ]:
data_stability = TestSuite(tests=[
    DataStability(),
])

data_stability.run(reference_data=adult_ref, current_data=adult_cur)
data_stability

In [ ]:
#test preset in a JSON format
data_stability.json()

In [ ]:
#test preset as a python object
data_stability.as_dict()

In [ ]:
data_quality = TestSuite(tests=[
    DataQuality(),
])

data_quality.run(reference_data=adult_ref, current_data=adult_cur)
data_quality

In [ ]:
data_drift = TestSuite(tests=[
    DataDrift(),
])

data_drift.run(reference_data=adult_ref, current_data=adult_cur)
data_drift

In [ ]:
no_target_performance = TestSuite(tests=[
    NoTargetPerformance(most_important_features=['education-num', 'hours-per-week']),
])

no_target_performance.run(reference_data=adult_ref, current_data=adult_cur)
no_target_performance

In [ ]:
regression_performance = TestSuite(tests=[
    Regression()
])

regression_performance.run(reference_data=housing_ref, current_data=housing_cur)
regression_performance

In [ ]:
classification_performance = TestSuite(tests=[
   MulticlassClassification(prediction_type='labels')
])

classification_performance.run(reference_data=iris_ref, current_data=iris_cur)
classification_performance

In [ ]:
binary_classification_performance = TestSuite(tests=[
    BinaryClassification(prediction_type='probas'),
])

binary_classification_performance.run(reference_data=bcancer_ref, current_data=bcancer_cur)
binary_classification_performance

In [ ]:
binary_topK_classification_performance = TestSuite(tests=[
    BinaryClassificationTopK(k=10),
])

binary_topK_classification_performance.run(reference_data=bcancer_ref, current_data=bcancer_cur)
binary_topK_classification_performance